# Train ruT5 for RUSSE Detox 2022

## Proprocess data

In [ ]:
!git clone https://github.com/skoltech-nlp/russe_detox_2022

Cloning into 'russe_detox_2022'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 82 (delta 22), reused 43 (delta 16), pack-reused 29
Unpacking objects: 100% (82/82), done.


In [ ]:
import re
import pandas as pd

# пока что мы объединяем dev и train: не хотим валидироваться на обучении, раз существует test-сет
data_df = pd.concat([
           pd.read_csv("./russe_detox_2022/data/input/dev.tsv", sep="\t"),
           pd.read_csv("./russe_detox_2022/data/input/train.tsv", sep="\t").drop(["index"], axis=1)
], axis=0).reset_index(drop=True)

# если у фразы несколько вариантов исправления — просто кладём их все как пары input-target
# исходим из того, что наши модели достаточно complex, чтобы это их не запутало
train_dict = {
    "input": [],
    "target": []
}

for tc, nc1, nc2, nc3 in zip(list(data_df["toxic_comment"]), list(data_df["neutral_comment1"]),
                             list(data_df["neutral_comment2"]), list(data_df["neutral_comment3"])):
  # здесь немножко препроцесса: из-за особенностей sentencepiece-токенизации модели плохо понимают
  # текст, написанный КАПСОМ. мы будем смотреть, если во входной строке больше 50% символов — капс,
  # и в таком случае приводить её к нижнему регистру
  input_str = str(tc)
  if len([c for c in input_str if re.search(r"[A-ZА-ЯЁ]", c)]) / len(input_str) > 0.5:
    input_str = input_str.lower()

  train_dict["input"].append(input_str)
  train_dict["target"].append(str(nc1))
  if type(nc2) != float: # проверка на NaN
    train_dict["input"].append(input_str)
    train_dict["target"].append(str(nc2))
  if type(nc3) != float: # проверка на NaN
    train_dict["input"].append(input_str)
    train_dict["target"].append(str(nc3))

train_df = pd.DataFrame(train_dict)

# перемешиваем датасет; задаём seed, чтобы результаты перемешивания совпадали между разными запусками
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# и всё, этого достаточно! токенизаторы у каждой модели свои, лемматизация и другая нормализация
# только уменьшат количество полезных данных для transformer-нейросетей

# пример того, что у нас в датафрейме:
train_df.sample(10)

,input,target
463,"а мне не сделал, вот хуй ты печальный :-(","А мне не сделал, вот ты печальный."
3089,мне скуучноо(( может опять твиты посвящать?,Мне скучно. Может опять твиты посвящать?
8116,"Отвалите от Волгограда, ублюдки, я не могу это...","Отстаньте от Волгограда, я не могу это читать"
12068,Каникулы. А я сука сижу дома из-за ебаной пого...,Каникулы. А я сижу дома из-за плохой погоды.
7599,С кем? С каким-то независимым оператором ? Что...,С кем? С каким-то независимым оператором? Что ...
5138,автор иди ты на хуй со своими классами деду он...,автор отстань со своими классами деду они и со...
5994,"только заметила, что измазала кровью всю дубле...","только заметила, что измазала кровью всю дубле..."
12146,"алгебра, литер и русск дописать!( а тебе?","Алгебру,литературу и русский язык дописать. А ..."
7503,да а я сегодня познакомилась нехотя с блядью мужа,Да я сегодня познакомилась нехотя с любовницей...
11317,спартак мы всеровно с тобой остальные пусть ху...,"Спартак мы все равно с тобой, остальные пускай..."


## Train ruT5 (Sber)

In [ ]:
# устанавливаем необходимые библиотеки
!pip install -qqq happytransformer sentencepiece

     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 17.3 MB/s 
     |████████████████████████████████| 325 kB 50.4 MB/s 
     |████████████████████████████████| 3.8 MB 42.6 MB/s 
     |████████████████████████████████| 67 kB 1.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.0 MB/s 
     |████████████████████████████████| 212 kB 52.5 MB/s 
     |████████████████████████████████| 134 kB 57.5 MB/s 
     |████████████████████████████████| 127 kB 50.4 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 6.5 MB 62.7 MB/s 
     |████████████████████████████████| 895 kB 58.8 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 54.0 MB/s 
     |████████████████████████████████| 271 kB 75.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [ ]:
import csv

# подготавливаем обучающие данные
train_df.to_csv("train.csv", index=False, quoting=csv.QUOTE_ALL)

In [ ]:
from happytransformer import HappyTextToText, TTTrainArgs
from transformers import AutoTokenizer

# берём модель ruT5 от Сбера, размер чекпойнта base
model = HappyTextToText("T5", "sberbank-ai/ruT5-base")

# 2 эпохи обучения
args = TTTrainArgs(num_train_epochs=2) 
model.train("train.csv", args=args)

03/20/2022 20:41:53 - INFO - happytransformer.happy_transformer -   Using model: cuda
03/20/2022 20:41:57 - INFO - happytransformer.happy_transformer -   Preprocessing training data...


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c151c153fba95462/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

03/20/2022 20:42:04 - INFO - happytransformer.happy_transformer -   Training...
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 12206
  Num Epochs = 2
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 24412


Step,Training Loss
500,3.392000
1000,2.270800
1500,2.053100
2000,2.023500
2500,1.917300
3000,1.904500
3500,1.889600
4000,1.812700
4500,1.702800
5000,1.721300




Training completed. Do not forget to share your model on huggingface.co/models =)




## Evaluate model

In [ ]:
from tqdm.notebook import tqdm

# подготавливаем тест сет
test_df = pd.read_csv("./russe_detox_2022/data/input/test.tsv", sep="\t")
output = [model.generate_text(entry) for entry in tqdm(list(test_df["toxic_comment"]))]
output[:20]

[TextToTextResult(text='Кто это придумывает? Кто это придумывает'),
 TextToTextResult(text='В такой ситуации виноваты люди из Ростелекома у которых даже кошка может купить фильм с пульта'),
 TextToTextResult(text='актёр может и не плохой, но как человек - не хороший'),
 TextToTextResult(text='Наказывать всех кто нарушает общественный порядок..'),
 TextToTextResult(text='Такие же люди и привели этих людей..'),
 TextToTextResult(text='А зачем тогда ты здесь это писал? А зачем ты здесь это писал?'),
 TextToTextResult(text='Главный плохой человек года. повар из полиции из миннеаполиса сварщик из бейрута президент минска из d'),
 TextToTextResult(text='Начни сваих людей в покое'),
 TextToTextResult(text='дайте уже пожить, создать семью отдал 35 лет жизникормил меня, а мой отец 60 лет хватит'),
 TextToTextResult(text='а ты помнишь, что 41 год помнишь? сколько ей денег заплатили, чтоб она такую чушь несла.'),
 TextToTextResult(text='С которым через час расстаешься и будешь заниматься любовью 

In [ ]:
# сохраняем предсказания, подготавливаем для кодалаба
with open("output_rut5_base_2.txt", "w") as out_file:
    out_file.write("\n".join([entry.text for entry in output]))

!zip output_rut5_base_2.zip output_rut5_base_2.txt

  adding: output_rut5_base_2.txt (deflated 68%)


## Export model

In [ ]:
# сохраняем модель
model.save("rut5_base_2/")
!tar -czf rut5_base_2.tar.gz rut5_base_2

Configuration saved in rut5_base_2/config.json
Model weights saved in rut5_base_2/pytorch_model.bin
tokenizer config file saved in rut5_base_2/tokenizer_config.json
Special tokens file saved in rut5_base_2/special_tokens_map.json
Copy vocab file to rut5_base_2/spiece.model


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# выгружаем веса на гугл диск

#!mkdir /content/drive/MyDrive/rudetox
!cp rut5_base_2.tar.gz /content/drive/MyDrive/rudetox
!cp output_rut5_base_2.zip /content/drive/MyDrive/rudetox